<img src="images/logo_city.png" align="right" width="20%">

# Data Generation Module Development

## 1. Goal

The module should be something like below:

In [1]:
import timeslice                    
import timeslice.source as source   
import timeslice.rule  as rule      
import timeslice.worker as worker
import timeslice.viz as viz # to be done

import torch

In [2]:
# set time split rule for dataset generation
time_rule = rule.TimeSlice(stp='2017-05-01 00:00:00', etp='2017-05-02 00:00:00', freq='10min')

# connect to database source
taxi_db = source.DatabaseSource('cleaned_small_yellow_2017_full', time_rule)

# initialize worker
data_worker = worker.Worker(source=taxi_db, destin='data_test/monthly_data/may', rule=time_rule, viz=True)

# generate dataset
# data_worker.generate()

AttributeError: 'DatabaseSource' object has no attribute 'pattern'

In [ ]:
time_rule = rule.TimeSlice(stp='2019-01-01 00:00:00', etp='2020-01-01 00:00:00', freq=f'W-MON')

time_rule.freq

time_rule.stp

time_rule.etp

time_rule.timesplit()

time_rule.tslices

In [ ]:
import pandas as pd
pd.date_range('2017-05-01 00:00:00', '2017-06-01 00:00:00', freq='D')

In [ ]:
d = {}

In [ ]:
import re
import pandas as pd
import pprint

def _construct_sql(stp:str, etp:str):
        '''
        A private helper function to construct sql query, called another
        helper function _construct_split.

        Args:
            stp: datetime string, starting time point of a concurrent unit
            etp: datatime string, end time point of a concurrent unit

        Returns:
            sql: a constructed query string
        '''
        pattern = re.compile(
            '^([0-9]{4})-([0-1][0-9])-([0-3][0-9])\s([0-1][0-9]|[2][0-3]):([0-5][0-9]):([0-5][0-9])$'
        )
        assert pattern.match(stp) and pattern.match(etp)

        return (f"select tripid,tpep_pickup_datetime,tpep_dropoff_datetime,pulocationid,dolocationid from cleaned_small_yellow_2017_full "
                f"where tpep_pickup_datetime >= {stp} and tpep_dropoff_datetime < {etp};")
    


bounds = pd.date_range('2017-05-01 00:00:00', '2017-06-01 00:00:00', freq='D')
subs = list(zip(bounds[:-1], bounds[1:]))
queries = {}

# create sub interval queries
for i, sub in enumerate(subs):
    stp, etp = list(map(str, sub))
    queries[i] = _construct_sql(stp, etp)

queries[0], queries[1]